Required Libraries

In [1]:
!pip install pandas scikit-learn

Libraries Import

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

Load All CSV Files

In [3]:
# Main training dataset
df = pd.read_csv("dataset.csv")

# Supporting files
severity = pd.read_csv("Symptom-severity.csv")
descriptions = pd.read_csv("symptom_Description.csv")
precautions = pd.read_csv("symptom_precaution.csv")

Data Cleaning

In [12]:
df = df.fillna(0)

# Remove leading/trailing spaces from column values
for col in df.columns[1:]:
    df[col] = df[col].astype(str).str.strip()

Get All Unique Symptoms

In [13]:
symptoms = {}
for i in df.columns[1:]:
    for j in set(df[i]):
        if j not in symptoms and j != '0':
            symptoms[j] = 1
        else:
            symptoms[j] += 1

symptom_list = sorted(symptoms.keys())

Convert Data to Binary Format(Symptoms as Features)

In [14]:
df1 = pd.DataFrame()
df1['Disease'] = df['Disease']
for sym in symptom_list:
    df1[sym] = 0

# Mark 1 where symptom exists
for i, row in df.iterrows():
    for col in df.columns[1:]:
        if row[col] != '0':
            df1.loc[i, row[col]] = 1

/tmp/ipython-input-14-3570169978.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[sym] = 0
/tmp/ipython-input-14-3570169978.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[sym] = 0
/tmp/ipython-input-14-3570169978.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[sym] = 0
/tmp/ip

Prepare Features And Labels

In [15]:
X = df1.drop(columns=['Disease']).values
y = df1['Disease'].values
le = LabelEncoder()
y_encoded = le.fit_transform(y)

Train Model (Random Forest Classifier)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.15, random_state=42)
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
print("Model Accuracy:", model.score(X_test, y_test))

Model Accuracy: 1.0


Train Model

In [17]:
import pickle

pickle.dump(model, open('disease_model.pkl', 'wb'))

Prediction Function (Top 5 Disease)

In [19]:
def predict_disease(input_symptoms):
    # Create input feature vector
    input_vector = [1 if sym in input_symptoms else 0 for sym in symptom_list]

    # Predict
    proba = model.predict_proba([input_vector])[0]
    top5_indices = np.argsort(proba)[-5:][::-1]
    top5_probs = proba[top5_indices]
    top5_diseases = le.inverse_transform(top5_indices)

    for i in range(5):
        disease = top5_diseases[i]
        prob = top5_probs[i]
        print(f"\n Prediction #{i+1}: {disease} — Probability: {prob:.2f}")

        # Description
        desc = descriptions[descriptions['Disease'] == disease]['Description']
        if not desc.empty:
            print(f"Description: {desc.values[0]}")

        # Precautions
        print("Precautions:")
        row = precautions[precautions['Disease'] == disease]
        if not row.empty:
            for j in range(1, 5):
                print(f"- {row.iloc[0][f'Precaution_{j}']}")
        print("—" * 40)

Testing Model

In [20]:
# Example Symptoms
user_input = ['headache', 'vomiting', 'chills']
predict_disease(user_input)


 Prediction #1: Paralysis (brain hemorrhage) — Probability: 0.36
Description: Intracerebral hemorrhage (ICH) is when blood suddenly bursts into brain tissue, causing damage to your brain. Symptoms usually appear suddenly during ICH. They include headache, weakness, confusion, and paralysis, particularly on one side of your body.
Precautions:
- massage
- eat healthy
- exercise
- consult doctor
————————————————————————————————————————

 Prediction #2: Allergy — Probability: 0.26
Description: An allergy is an immune system response to a foreign substance that's not typically harmful to your body.They can include certain foods, pollen, or pet dander. Your immune system's job is to keep you healthy by fighting harmful pathogens.
Precautions:
- apply calamine
- cover area with bandage
- nan
- use ice to compress itching
————————————————————————————————————————

 Prediction #3: Malaria — Probability: 0.12
Description: An infectious disease caused by protozoan parasites from the Plasmodium fa